<a href="https://colab.research.google.com/github/ChanHyeokKong/Tourist/blob/main/Tourlist_with_openai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai requests

In [ ]:
!pip uninstall openai

In [ ]:
!pip install openai==0.28

In [12]:
import os

gmaps_key = 'Enter the Google Maps API key here'

os.environ['OPENAI_API_KEY'] = 'Enter the OpenAI API key here' # OpenAI api key를 입력하세요(문자열)

client = openai.OpenAI()

In [ ]:
import openai
import requests
import time

# OpenAI API 키 설정
openai.api_key = open_key  # 실제 API 키로 대체

# 구글 맵 API 키 설정
google_maps_api_key = gmaps_key  # 실제 API 키로 대체

def fetch_place_ids(city_name):
    search_url = f'https://maps.googleapis.com/maps/api/place/textsearch/json?query=tourist+attractions+in+{city_name}&key={google_maps_api_key}'
    response = requests.get(search_url)
    search_results = response.json()

    print("Search Results:", search_results)  # 디버깅: 검색 결과 확인

    place_ids = []
    if 'results' in search_results:
        for place in search_results['results']:
            place_ids.append(place['place_id'])

    return place_ids

def fetch_place_reviews(place_id):
    url = f'https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&fields=name,rating,reviews&key={google_maps_api_key}'
    response = requests.get(url)
    place_details = response.json()

    print(f"Place Details for {place_id}:", place_details)  # 디버깅: 장소 세부 정보 확인

    if 'result' not in place_details:
        return None

    reviews = place_details['result'].get('reviews', [])
    return reviews

def generate_travel_plan(reviews, user_request):
    review_texts = " ".join([review['text'] for review in reviews])
    prompt = f"""
    다음은 관광 명소 및 레스토랑에 대한 리뷰입니다:

    {review_texts}

    사용자는 "{user_request}"에 중점을 둔 여행 계획을 원합니다.

    제공된 리뷰를 바탕으로 여행 계획을 자세히 제시해 주세요.
    가장 평가가 높은 장소와 그들의 특징을 강조해 주세요.
    """

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # 최신 모델을 사용할 수 있습니다.
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500,
        temperature=0.7
    )

    return response.choices[0].message.content

def main():
    # 사용자 입력 받기
    city_name = input("도시를 입력하세요 (예: 타이페이): ")
    user_request = input("여행 관심사를 입력하세요 (예: 역사적인 관광지): ")

    if not city_name or not user_request:
        print("도시와 여행 관심사가 필요합니다.")
        return

    # 도시를 기반으로 장소 ID 검색
    place_ids = fetch_place_ids(city_name)

    print("Place IDs:", place_ids)  # 디버깅: 장소 ID 확인

    if not place_ids:
        print("장소를 찾을 수 없습니다.")
        return

    # 리뷰 수집
    all_reviews = []
    for place_id in place_ids:
        reviews = fetch_place_reviews(place_id)
        if reviews:
            all_reviews.extend(reviews)

    print("All Reviews:", all_reviews)  # 디버깅: 모든 리뷰 확인

    if not all_reviews:
        print("리뷰를 불러오는 데 실패했습니다.")
        return

    # 여행 계획 생성
    travel_plan = generate_travel_plan(all_reviews, user_request)

    # 결과 출력
    print("\n여행 계획:")
    print(travel_plan)

    time.sleep(20)

if __name__ == "__main__":
    main()


RAG 에러 확인 필요
Chat GPT와 실행결과 비교를 통해 리뷰를 사용하고 있는지 비교 필요